In [ ]:
# Import packages
# Make sure you have RiverREM imported, check readme for instructions
import os
import pathlib

import earthpy as et
import earthpy.plot as ep
import matplotlib.pyplot as plt
from load_plot_model import load_dtm, plot_model, plot_hists, plot_hist, run_rem_maker
from riverrem.REMMaker import REMMaker, clear_osm_cache
import requests
import rioxarray as rxr

In [ ]:
# Set working directory
working_dir = os.path.join(
    pathlib.Path.home(), 'earth-analytics', 'data', 'watershed-project')

# Try/Except Block   
try:
    os.chdir(working_dir)
except:
    print('{} does not exist. Creating...'.format(working_dir))
    os.makedirs(working_dir)
    os.chdir(working_dir)
else:
    print('{} is now the working directory'.format(working_dir))

# Load the UAV DTMs and REMs

In [ ]:
# Define urls to UAV DTMs and REMs (saved on zenodo)
# make into function
highway93_dtm_url = ('https://zenodo.org/record/8218054/files/'
                     'highway93_uav_dtm.tif?download=1')
highway93_rem_url = ('https://zenodo.org/record/8218054/files/'
                     'highway93_uav_rem.tif?download=1')
highway93_lidar_dtm_url = ('https://zenodo.org/record/8218054/files/'
                           'highway93_lidar_dtm.asc?download=1')
applevalley_dtm_url = ('https://zenodo.org/record/8218054/files/'
                       'applevalley_uav_dtm.tif?download=1')
applevalley_rem_url = ('https://zenodo.org/record/8218054/files/'
                       'applevalley_uav_rem.tif?download=1')
applevalley_lidar_dtm_url = ('https://zenodo.org/record/8218054/files/'
                             'applevalley_lidar_dtm.asc?download=1') 
hallmeadows_dtm_url = ('https://zenodo.org/record/8218054/files/'
                       'hallmeadows_uav_dtm.tif?download=1')
hallmeadows_rem_url = ('https://zenodo.org/record/8218054/files/'
                       'hallmeadows_uav_rem.tif?download=1')
hallmeadows_lidar_dtm_url = ('https://zenodo.org/record/8218054/files/'
                             'hallmeadows_lidar_dtm.asc?download=1')

In [ ]:
# Load dataarrays for UAV-derived dtms and rems and lidar rems
highway93_uav_dtm = load_dtm(data_url = highway93_dtm_url, 
                          site_name = 'highway93',
                          file_name = 'highway93_dtm.tif')
applevalley_uav_dtm = load_dtm(data_url = applevalley_dtm_url, 
                          site_name = 'applevalley',
                          file_name = 'applevalley_dtm.tif')
hallmeadows_uav_dtm = load_dtm(data_url = hallmeadows_dtm_url, 
                          site_name = 'hallmeadows',
                          file_name = 'hallmeadows_dtm.tif')
highway93_uav_rem = load_dtm(data_url = highway93_rem_url, 
                          site_name = 'highway93',
                          file_name = 'highway93_rem.tif')
applevalley_uav_rem = load_dtm(data_url = applevalley_rem_url, 
                          site_name = 'applevalley',
                          file_name = 'applevalley_rem.tif')
hallmeadows_uav_rem = load_dtm(data_url = hallmeadows_rem_url, 
                          site_name = 'hallmeadows',
                          file_name = 'hallmeadows_rem.tif')
highway93_lidar_dtm = load_dtm(data_url = highway93_lidar_dtm_url, 
                          site_name = 'highway93',
                          file_name = 'highway93_lidar_dtm.asc')
applevalley_lidar_dtm = load_dtm(data_url = applevalley_lidar_dtm_url, 
                          site_name = 'applevalley',
                          file_name = 'applevalley_lidar_dtm.asc')
hallmeadows_lidar_dtm = load_dtm(data_url = hallmeadows_lidar_dtm_url, 
                          site_name = 'hallmeadows',
                          file_name = 'hallmeadows_lidar_dtm.asc')

# Lists of UAV DTMs and REMs
uav_dtm_list = [highway93_uav_dtm, 
                applevalley_uav_dtm, 
                hallmeadows_uav_dtm]

uav_rem_list = [highway93_uav_rem, 
                applevalley_uav_rem, 
                hallmeadows_uav_rem]

lidar_dtm_list = [highway93_lidar_dtm,
                 applevalley_lidar_dtm,
                 hallmeadows_lidar_dtm]

In [ ]:
#lidar_crs = highway93_lidar_dtm_local.rio.crs
lidar_dtm_path_list = [highway93_lidar_dtm_url, 
                      applevalley_lidar_dtm, 
                      hallmeadows_lidar_dtm]

# Process LiDAR Data

In [ ]:
def reproject_match_lidar(site_uav_rem, lidar_path, lidar_crs, site_name):
    """
    Loads the lidar DTM, reprojects and matches the resolution and boundary of the UAV REM. 
    Saves as a tif file for use in RiverREM
    
    Parameters
    ------------   
    site_uav_rem: dataarray
        The rem to reproject and match.

    lidar_path: str
        Path to LiDAR dtm.
        
    lidar_path: int
        Original crs of the LiDAR dtm.
        
    Returns
    -------
    lidar_dtm_match: dataarray.
        The processed lidar dtm.
    """
    # Create dataarray for lidar DTM
    lidar_dtm = rxr.open_rasterio(lidar_path, masked=True)
    
    # Write CRS to lidar_dtm (2876); remove this step once link updated
    lidar_dtm_crs = lidar_dtm.rio.write_crs(lidar_crs) 
    
    # Reproject/match lidar DTM to UAV REM
    lidar_dtm_match = lidar_dtm_crs.rio.reproject_match(site_uav_rem)
    
    # Save the clipped lidar dtm as raster for use in RiverREM
    #lidar_dtm_match.rio.to_raster(site_name, ("{}_lidar_dtm_clipped.tif").format(site_name))
    
    return lidar_dtm_match

In [ ]:
def clip_lidar(site_uav_rem, lidar_path, lidar_crs, site_name):
    """Short function to clip lidar for draft blog post"""
    # Create dataarray for lidar DTM
    lidar_dtm = rxr.open_rasterio(lidar_path, masked=True)
    
    # Write CRS to lidar_dtm (2876); remove this step once link updated
    lidar_dtm_crs = lidar_dtm.rio.write_crs(lidar_crs) 
    
    # Reproject lidar DTM to UAV REM CRS
    lidar_dtm_reproject = lidar_dtm_crs.rio.reproject(site_uav_rem)
    
    # Boundaries of UAV REM
    uav_rem_bounds = site_uav_rem.rio.bounds()
    uav_rem_crs = site_uav_rem.rio.crs

    lidar_clipped = lidar_dtm_reproject.rio.clip(uav_rem_bounds, uav_rem_crs)
    
    #Clip to UAV REM extent
    
    return lidar_clipped

In [ ]:
# Load clipped dtm for example site for blog post
applevalley_lidar_dtm_clipped = clip_lidar(site_uav_rem = applevalley_uav_rem,
                                           lidar_path =applevalley_lidar_dtm_url, 
                                           lidar_crs = 2876,
                                           site_name='applevalley')

In [ ]:
plot_hist(model=applevalley_lidar_dtm_matched, title='apple valley histogram', color='cyan')

In [ ]:
# Run the process lidar function to create a lidar-derived DTM for each site
site_names = ('highway93', 'applevalley', 'hallmeadows')

# How to use .format with a non-string object?
#lidar_dtm_list = []
#for site in site_names:
 #   processed_lidar_dtm = reproject_match_lidar(site_uav_rem, lidar_path, lidar_crs, name)
    #lidar_dtm_list.append(lidar_rem)
    

# Just the LiDAR DTM Plots

In [ ]:
# Define plot space with x subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# List of site names to pass to function for title
plot_titles = ('Highway 93', 'Apple Valley', 'Hall Meadows')

# List of dtms to pass to function
lidar_dtm_list

# Call plot function
for i, axe in enumerate(axes.flatten()):
    plot_models(
        model=lidar_dtm_list[i],
        title=plot_titles[i],
        coarsen=(False, False, False),
        ax=axe)

# Create LiDAR REMs Using REMMaker Tool

In [ ]:
def run_rem_maker(name, k=100):
    """Run the REMMaker tool"""
    
    # Input the DTM file path and desired output directory
    override_cache = False
    lidar_dtm_path = os.path.join(('LiDAR_DTM_{}.tif').format(name))
    lidar_out_path = os.path.join(lidar_dir, ('lidar_{}').format(name))
    lidar_rem_path = os.path.join(lidar_out_path, 
                                  ('LiDAR_DTM_{}_REM.tif').format(name))

    # Run the REMMaker if the path to the REM does not already exist
    if (not os.path.exists(lidar_rem_path)) or override_cache:
        rem_maker = REMMaker(dem=lidar_dtm_path, 
                             out_dir=lidar_out_path, 
                             interp_pts=1000, 
                             k=100)

        # clear OSM cache
        clear_osm_cache()

        # create an REM
        rem_maker.make_rem()

        # create an REM visualization with the given colormap
        rem_maker.make_rem_viz(cmap='mako_r')

    else:
        print('The REM already exists. Not running REMMaker')

In [ ]:
# Create REM for all sites with run_rem_maker function
site_names = ('highway93', 'applevalley', 'hallmeadows')
lidar_rem_list = []

for site in site_names:
    run_rem_maker(name=site)
    lidar_rem_path = os.path.join(lidar_dir,
                                  ('lidar_{}').format(site),
                                  ('LiDAR_DTM_{}_REM.tif').format(site))
    lidar_rem = rxr.open_rasterio(lidar_rem_path, masked=True)    
    lidar_rem_list.append(lidar_rem)

# Plot the UAV and LiDAR Derived REMs

In [ ]:
# Plot the REMs
# Define plot space with x subplots
fig, axes = plt.subplots(3, 2, figsize=(12, 14))

# List of site names to pass to function for titles
plot_title = ('UAV-Derived REM for Highway 93', 
              'UAV-Derived REM for Apple Valley', 
              'UAV-Derived REM for Hall Meadows',
              'LiDAR-Derived REM for Highway 93', 
              'LiDAR-Derived REM for Apple Valley', 
              'LiDAR-Derived REM for Hall Meadows')

# List of REMs to pass to plot function
all_rem_list = (uav_rem_list + lidar_rem_list)

# Call plot function
for i, axe in enumerate(axes.flatten()):
    plot_models(
        model=all_rem_list[i],
        title=plot_title[i],
        coarsen=(True, True, True, False, False, False),
        ax=axe)
# Note - I increased value of k in the REMMAker and streaking improved!!
# "param k: number of nearest neighbors to use for IDW interpolation. If None, an appropriate value is estimated.
#  The estimation routine uses k between 5-100 points (0.5-10% of the river length) depending on the
#  sinuosity of the river of interest. Greater values of k are used for more sinuous rivers.""